# Notebook 02: Building a Simple Agent with Tools

## 🎯 What is This Notebook About?

Welcome to Notebook 02! This notebook shows you how to build autonomous agents with **client-side tools** - Python functions that agents can use to take actions.

**What we'll do:**
- Create a Wikipedia search tool (simple example, no API key needed)
- Build an agent that uses tools to answer questions
- Create IT operations tools for managing infrastructure
- See how agents automatically detect tools from function docstrings

**Why this matters:** Tools give agents the ability to act, not just think. Client-side tools are simple Python functions with docstrings - no complex configuration needed!

**The big picture:** In Notebook 01, we saw how agents reason. Now we'll give agents "hands" - tools they can use to take actions. Client-side tools run in your Python process, making them fast and secure for prototyping.

---

## 📚 Key Concepts Explained

### Client-Side Tools

**What they are:** Python functions with docstrings that agents can call to perform actions. They run in your Python process, not on a remote server.

**Why they matter:** Tools give agents the ability to act. Without tools, agents can only think and respond - with tools, they can actually do things!

**Think of it like:** An agent without tools = a consultant who can only give advice. An agent with tools = a consultant who can actually fix things.

**How it works:** You write a Python function with a clear docstring → The agent reads the docstring → When needed, the agent calls the function → Function executes and returns results → Agent uses results to complete the task.

### Docstrings: The Secret Sauce

**What they are:** Text descriptions in triple quotes that explain what a function does, what parameters it takes, and when to use it.

**Why they matter:** The agent doesn't see your code - it only sees the docstring. A good docstring makes the difference between a tool the agent uses correctly and one it ignores.

**Think of it like:** A menu description. A good description makes you want to order the dish and tells you what to expect.

### Tool Registry & Sessions

**Tool Registry:** A dictionary that maps tool names to functions (`{"tool_name": function}`). It tells the agent what tools are available.

**Sessions:** Maintain conversation context - agents remember what happened in previous turns, enabling multi-step operations.

**Think of it like:** Registry = a toolbox (list of available tools). Session = a phone call (maintains context throughout).

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand what client-side tools are and how they work
- ✅ Know how to create tools using Python functions with docstrings
- ✅ See how agents automatically detect and use tools
- ✅ Build an agent that uses tools to answer questions
- ✅ Create IT operations tools for infrastructure management
- ✅ Understand the difference between client-side tools and MCP tools (covered in Notebook 04)

---

## ⚠️ Prerequisites

Before starting this notebook, make sure you have:
- ✅ Completed Notebook 01: Introduction to Agents (understanding of agents and the agent loop)
- ✅ LlamaStack server running (see Module README)
- ✅ Python environment with dependencies installed (`llama-stack-client`, `requests`)
- ✅ Basic understanding of Python functions and docstrings

**The fun part:** No MCP servers needed yet! We're building client-side tools that run in your Python process.


---

## 📋 Step-by-Step Guide

### Step 1: Setup and Configuration

**What we're doing:** Setting up the environment and connecting to LlamaStack.

**Why:** We need to establish connections before we can build agents with tools.

**What to expect:**
- Import required libraries
- Load configuration from shared config system
- Connect to LlamaStack server
- Verify everything is working


In [ ]:
# Import required libraries
import sys
from pathlib import Path
import os
import requests
import json
import re
from termcolor import cprint

# Add src directory to path for shared configuration
root_dir = Path("../..").resolve()
sys.path.insert(0, str(root_dir / "src"))

# Import centralized configuration
from config import LLAMA_STACK_URL, MODEL, CONFIG

# Import LlamaStack client
from llama_stack_client import LlamaStackClient

print("✅ Libraries imported successfully!")
print(f"📡 LlamaStack URL: {LLAMA_STACK_URL}")
print(f"🤖 Model: {MODEL}")

# Verify configuration
if not LLAMA_STACK_URL:
    raise ValueError(
        "LLAMA_STACK_URL is not configured!\n"
        "Please run: ./scripts/setup-env.sh"
    )

# Initialize LlamaStack client
client = LlamaStackClient(base_url=LLAMA_STACK_URL)

# Verify connection
try:
    models = client.models.list()
    model_count = len(models.data) if hasattr(models, 'data') else len(models)
    print(f"\n✅ Connected to LlamaStack")
    print(f"   Available models: {model_count}")
except Exception as e:
    print(f"\n❌ Cannot connect to LlamaStack: {e}")
    raise

# Use MODEL from config
model = MODEL


**What happened:** After running the code, you should see successful connections to LlamaStack. The configuration is loaded from the shared `src/config.py` system, which auto-detects your environment.

**Key takeaway:** The shared configuration system makes it easy to switch between environments (local, OpenShift, etc.) without changing code.


### Step 2: Creating a Wikipedia Search Tool

**What we're doing:** Creating a custom Wikipedia search tool - a simple Python function that agents can use.

**Why:** Wikipedia is perfect for learning because:
- ✅ No API key required (free and easy!)
- ✅ Runs client-side (in your Python process - no external dependencies)
- ✅ Shows the pattern for creating any tool
- ✅ Perfect for factual information and general knowledge

**What to expect:**
- We'll create a `wikipedia_search` function with a clear docstring
- The docstring tells the agent what the tool does and when to use it
- We'll add it to a tool registry so the agent can find it
- No complex configuration needed - just a function with a good docstring!


In [ ]:
# Create the Wikipedia search function
# The docstring is critical - the agent reads it to understand the tool!

def wikipedia_search(query: str, max_results: int = 3) -> str:
    """
    Search Wikipedia for information.
    
    This tool searches Wikipedia for factual information. Use it when you need
    general knowledge, definitions, or information about well-known topics.
    
    :param query: The search query to look up (e.g., "Python programming", "Artificial Intelligence")
    :param max_results: Maximum number of results to return (default: 3)
    :return: A formatted string with Wikipedia search results
    """
    try:
        # First, try the search API (more flexible for general queries)
        search_url = "https://en.wikipedia.org/api/rest_v1/page/search"
        params = {"q": query, "limit": max_results}
        search_response = requests.get(search_url, params=params, timeout=5)
        
        if search_response.status_code == 200:
            results = search_response.json().get("pages", [])
            if results:
                formatted = f"Wikipedia search results for '{query}':\n\n"
                for j, page in enumerate(results[:max_results], 1):
                    title = page.get("title", "No title")
                    snippet = page.get("snippet", "No snippet")
                    # Try to get the full summary for the first result
                    if j == 1:
                        try:
                            page_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title.replace(' ', '_')}"
                            page_response = requests.get(page_url, timeout=5)
                            if page_response.status_code == 200:
                                page_data = page_response.json()
                                extract = page_data.get("extract", snippet)
                                formatted += f"{j}. {title}\n"
                                formatted += f"   {extract[:400]}...\n\n"
                            else:
                                formatted += f"{j}. {title}\n"
                                formatted += f"   {snippet[:300]}...\n\n"
                        except:
                            formatted += f"{j}. {title}\n"
                            formatted += f"   {snippet[:300]}...\n\n"
                    else:
                        formatted += f"{j}. {title}\n"
                        formatted += f"   {snippet[:300]}...\n\n"
                return formatted.strip()
        
        # Fallback: try direct page access
        page_url = "https://en.wikipedia.org/api/rest_v1/page/summary/" + query.replace(" ", "_")
        response = requests.get(page_url, timeout=5)
        
        if response.status_code == 200:
            data = response.json()
            title = data.get("title", query)
            extract = data.get("extract", "No summary available")
            page_url_desktop = data.get("content_urls", {}).get("desktop", {}).get("page", "")
            
            result = f"Wikipedia: {title}\n"
            result += f"URL: {page_url_desktop}\n"
            result += f"Summary: {extract[:500]}...\n"
            return result
            
        return f"No Wikipedia results found for: {query}"
            
    except Exception as e:
        return f"Error searching Wikipedia: {str(e)}"

print("✅ Wikipedia search function created!")
print("   Function: wikipedia_search(query: str, max_results: int = 3)")
print("   💡 The docstring tells the agent what this tool does and when to use it")


**What happened:** After running the code, you created a Wikipedia search function. Notice the docstring - it describes what the function does, what parameters it takes, and when to use it. This docstring is what the agent reads to understand the tool!

**Key takeaway:** Docstrings are critical for tools. The agent doesn't see your code - it only sees the docstring. A clear, descriptive docstring helps the agent understand when and how to use the tool.


### Step 3: Creating a Tool Registry and System Prompt

**What we're doing:** Adding the tool to a registry and creating a system prompt that tells the agent about available tools.

**Why:** The agent needs to know what tools are available. We create a tool registry (dictionary) and a system prompt that describes the tools.

**What to expect:**
- Create a tool registry (dictionary mapping tool names to functions)
- Create a system prompt that describes available tools
- Understand how the agent discovers tools from the prompt


In [ ]:
# Helper function to parse JSON tool calls from response
def parse_tool_call(content):
    """Parse JSON tool call from model response."""
    content = content.strip()
    
    # Try to find JSON object
    json_match = re.search(r'\{[^{}]*"tool_call"[^{}]*\}', content)
    if not json_match:
        # Try broader match for nested objects
        brace_count = 0
        start_idx = content.find('{')
        if start_idx != -1:
            for i in range(start_idx, len(content)):
                if content[i] == '{':
                    brace_count += 1
                elif content[i] == '}':
                    brace_count -= 1
                    if brace_count == 0:
                        json_str = content[start_idx:i+1]
                        try:
                            return json.loads(json_str)
                        except:
                            pass
    
    if json_match:
        try:
            return json.loads(json_match.group())
        except:
            pass
    
    return None

# Create tool registry - maps tool names to functions
tool_registry = {"wikipedia_search": wikipedia_search}

# Create system prompt describing the tool
wikipedia_tool_description = """- wikipedia_search(query: str, max_results: int = 3): Search Wikipedia for information. 
  Use this tool when you need general knowledge, definitions, or information about well-known topics.
  Parameters:
    - query (string): The search query to look up (e.g., "Python programming", "Artificial Intelligence")
    - max_results (integer): Maximum number of results to return (default: 3)"""

system_prompt = f"""You are a helpful assistant with access to tools.

When you need to use a tool, respond ONLY with valid JSON in this exact format:
{{
  "tool_call": "function_name",
  "arguments": {{"param1": "value1", "param2": "value2"}}
}}

Available tools:
{wikipedia_tool_description}

IMPORTANT: 
- If you need to use a tool, respond with ONLY the JSON object, nothing else.
- Do not include any text before or after the JSON.
- Use the exact function name as shown above."""

print("✅ Tool registry created!")
print(f"   Tools available: {list(tool_registry.keys())}")
print("\n✅ System prompt created!")
print("   💡 The system prompt tells the agent what tools are available and how to use them")


**What happened:** After running the code, you created a tool registry (dictionary) that maps tool names to functions, and a system prompt that describes the tools to the agent.

**Key takeaway:** The tool registry is how you tell the agent what tools are available. The system prompt describes the tools in a way the agent can understand. Together, they enable the agent to discover and use tools.


### Step 4: Creating a Helper Function for Tool Execution

**What we're doing:** Creating a helper function that handles the complete flow: sending a query to the agent, detecting tool calls, executing tools, and getting the final response.

**Why:** This helper function simplifies the process of interacting with agents that use tools. It handles all the complexity of detecting tool calls, executing them, and managing the conversation flow.

**What to expect:**
- Create a function that executes turns with tool detection
- See how tool calls are detected from agent responses
- Understand how tools are executed and results are integrated


In [ ]:
# Helper function to execute turns with tool detection
def execute_turn(task, messages, tool_registry, max_tokens=200):
    """
    Execute a turn with the agent, detecting and executing tool calls.
    
    Returns True if a tool was used, False otherwise.
    """
    print(f"\n📋 Task: {task}")
    print(f"\n🔄 Agent execution:")
    print("-" * 60)
    
    cprint(f"User> {task}", "green")
    
    # Add user message to conversation
    messages.append({
        "role": "user",
        "content": task,
    })
    
    # Create chat completion with streaming
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
        max_tokens=max_tokens,
        temperature=0.1  # Lower temperature for more consistent JSON output
    )
    
    # Process the stream
    full_response = ""
    print("\n🤖 Agent response:")
    print("-" * 60)
    for chunk in stream:
        if chunk.choices and len(chunk.choices) > 0:
            delta = chunk.choices[0].delta
            if delta.content:
                content = delta.content
                print(content, end="", flush=True)
                full_response += content
    
    print("\n\n" + "-" * 60)
    
    # Check if response contains a tool call
    tool_call_data = parse_tool_call(full_response)
    tool_calls_found = False
    
    if tool_call_data and "tool_call" in tool_call_data:
        tool_calls_found = True
        func_name = tool_call_data["tool_call"]
        func_args = tool_call_data.get("arguments", {})
        
        print(f"\n🔧 Tool call detected: {func_name}")
        print(f"   Arguments: {func_args}")
        
        # Execute the tool
        if func_name in tool_registry:
            print(f"\n   → Executing {func_name}...")
            try:
                result = tool_registry[func_name](**func_args)
                print(f"   ✅ Tool result received ({len(result)} chars)")
                
                # Add assistant message with tool call
                messages.append({
                    "role": "assistant",
                    "content": full_response
                })
                
                # Add tool result to conversation
                messages.append({
                    "role": "user",
                    "content": f"Tool result: {result}. Now provide a natural language response based on this information."
                })
                
                # Get final response with tool results
                print("\n🤖 Final response with tool results:")
                print("-" * 60)
                final_response = client.chat.completions.create(
                    model=model,
                    messages=messages,
                    stream=True,
                    max_tokens=300
                )
                
                final_text = ""
                for chunk in final_response:
                    if chunk.choices and len(chunk.choices) > 0:
                        delta = chunk.choices[0].delta
                        if delta.content:
                            content = delta.content
                            print(content, end="", flush=True)
                            final_text += content
                
                # Add final assistant response to messages
                messages.append({
                    "role": "assistant",
                    "content": final_text
                })
            except Exception as e:
                print(f"   ❌ Error executing tool: {e}")
                tool_calls_found = False
        else:
            print(f"   ⚠️  Tool '{func_name}' not found in registry")
            tool_calls_found = False
    else:
        # No tool call detected - model answered directly
        messages.append({
            "role": "assistant",
            "content": full_response
        })
        print("\n💡 Model responded directly without using tools")
    
    print(f"\n\n{'=' * 60}")
    if tool_calls_found:
        print("✅ Turn completed! (Tools were used)")
    else:
        print("⚠️  Turn completed, but no tool calls detected")
    print(f"{'=' * 60}")
    
    return tool_calls_found

print("✅ Helper function created!")
print("   Function: execute_turn(task, messages, tool_registry)")
print("   💡 This function handles the complete flow: query → tool detection → execution → response")


**What happened:** After running the code, you created a helper function that handles the complete agent interaction flow. This function detects tool calls, executes tools, and manages the conversation.

**Key takeaway:** The helper function simplifies agent interactions. It handles tool detection, execution, and response generation automatically, so you can focus on building tools rather than managing the interaction flow.


### Step 5: Initializing a Conversation Session

**What we're doing:** Creating a conversation session with the agent.

**Why:** Sessions maintain conversation context. The agent remembers what happened in previous turns, making multi-step operations possible.

**What to expect:**
- Initialize a messages list with the system prompt
- The system prompt describes available tools
- This session will maintain context throughout our interactions


In [ ]:
# Initialize conversation session
# We'll use a messages list to maintain conversation context

messages = [
    {"role": "system", "content": system_prompt}
]

print("✅ Conversation session initialized!")
print("   💡 The system prompt describes available tools to the agent")
print("   💡 The messages list will maintain conversation history")


**What happened:** After running the code, you created a conversation session. The messages list starts with the system prompt, which tells the agent what tools are available.

**Key takeaway:** Sessions maintain context. As you add messages (user queries, agent responses, tool results), the agent remembers everything that happened, enabling multi-step operations.


### Step 6: Testing the Agent with Wikipedia Search

**What we're doing:** Testing the agent by asking it a question that requires using the Wikipedia search tool.

**Why:** This demonstrates the complete flow - the agent receives a query, decides to use a tool, calls the tool, gets results, and formulates a response.

**What to expect:**
- Send a query to the agent
- Watch the agent detect that it needs to use Wikipedia search
- See the tool execute and return results
- See the agent use those results to answer the question


In [ ]:
# Test the agent with a query that requires Wikipedia search
task = "What is Python programming? Use wikipedia_search to find information."

print("=" * 60)
print("Testing Agent with Wikipedia Search")
print("=" * 60)

# Execute the turn
tool_used = execute_turn(task, messages, tool_registry)

print("\n" + "=" * 60)
print("Turn Summary")
print("=" * 60)

if tool_used:
    print("\n✅ Tool was used successfully!")
    print("   💡 Look for these indicators in the output:")
    print("      🔧 = Tool call detected")
    print("      → = Tool being executed")
    print("      ✅ = Tool result received")
else:
    print("\n💡 No tool call detected")
    print("   The model may have answered directly, or tool call format was different")


**What happened:** After running the code, the agent received your query, detected that it needed to use Wikipedia search, called the tool, received results, and formulated a response using those results.

**Key takeaway:** This demonstrates the complete agent loop with tools: observe (receive query) → think (decide to use tool) → act (call tool) → learn (use results to answer). The agent seamlessly integrated tool results into its response.


### Step 7: Creating IT Operations Tools

**What we're doing:** Creating custom IT operations tools - functions that agents can use to manage infrastructure.

**Why:** This demonstrates how to build production-ready tools. We'll create tools for checking service status, restarting services, and getting system overviews.

**What to expect:**
- Create a simulated IT environment
- Build tools for service management
- See how agents use multiple tools in sequence
- Understand how to build tools for your own infrastructure


In [ ]:
# Create a simulated IT environment
# This simulates real IT infrastructure for safe testing

class SimpleITEnvironment:
    """Simple simulated IT environment for our custom tools."""
    def __init__(self):
        self.services = {
            "web-server": {"status": "online", "cpu": 45, "memory": 60},
            "database": {"status": "online", "cpu": 30, "memory": 50},
            "cache-service": {"status": "degraded", "cpu": 85, "memory": 90},
        }
    
    def get_service_status(self, service_name: str) -> dict:
        """Get service status."""
        return self.services.get(service_name, {"status": "not_found"})
    
    def restart_service(self, service_name: str) -> str:
        """Restart a service."""
        if service_name in self.services:
            self.services[service_name]["status"] = "online"
            self.services[service_name]["cpu"] = 20
            self.services[service_name]["memory"] = 30
            return f"Service {service_name} restarted successfully"
        return f"Service {service_name} not found"

# Initialize environment
env = SimpleITEnvironment()
print("✅ Created simulated IT environment")
print(f"   Services: {list(env.services.keys())}")
print("   💡 This is a safe simulated environment - perfect for learning!")

**What happened:** After running the code, you created a simulated IT environment with multiple services. Each service has status, CPU usage, and memory usage - just like real services.

**Key takeaway:** Simulated environments let you test agents safely before deploying to production. This is the same pattern we used in Notebook 01 - safe experimentation before real deployment.


### Step 8: Creating IT Operations Tools

**What we're doing:** Creating Python functions that agents can use to manage IT infrastructure.

**Why:** These tools demonstrate how to build production-ready tools. The same pattern works for any infrastructure - monitoring systems, service management, databases, anything!

**What to expect:**
- Create functions with clear docstrings
- Each function performs a specific IT operation
- The docstrings tell the agent when and how to use each tool
- We'll add these tools to a registry


In [ ]:
# Define IT operations tools as Python functions
# Remember: The docstrings are critical - the agent reads them!

def check_service_status(service_name: str) -> str:
    """
    Check the status of an IT service.
    
    Returns information about service health including CPU usage, memory usage, and current status.
    Use this to monitor service health and detect issues.
    
    :param service_name: The name of the service to check (e.g., 'web-server', 'database', 'cache-service')
    :return: A string describing the service status, CPU usage, and memory usage
    """
    status = env.get_service_status(service_name)
    if status.get("status") == "not_found":
        return f"Service '{service_name}' not found"
    
    return (
        f"Service: {service_name}\n"
        f"Status: {status['status']}\n"
        f"CPU Usage: {status['cpu']}%\n"
        f"Memory Usage: {status['memory']}%"
    )


def restart_service(service_name: str) -> str:
    """
    Restart an IT service that is not working properly.
    
    Use this when a service has failed or is in a degraded state.
    This will stop and start the service, which may cause brief downtime.
    
    :param service_name: The name of the service to restart (e.g., 'web-server', 'database')
    :return: A string confirming the restart operation
    """
    return env.restart_service(service_name)


def get_all_services() -> str:
    """
    Get the status of all services in the environment.
    
    Use this to get an overview of the entire system health.
    Returns a list of all services with their current status and metrics.
    
    :return: A string listing all services and their status
    """
    result = "All Services Status:\n"
    for name, status in env.services.items():
        result += f"\n{name}:\n"
        result += f"  Status: {status['status']}\n"
        result += f"  CPU: {status['cpu']}%\n"
        result += f"  Memory: {status['memory']}%\n"
    return result

print("✅ IT operations tools created!")
print("   - check_service_status(service_name: str)")
print("   - restart_service(service_name: str)")
print("   - get_all_services()")
print("\n💡 Notice the docstrings - they describe what each tool does and when to use it!")

**What happened:** After running the code, you created three IT operations tools. Each function has a clear docstring that describes what it does, when to use it, and what parameters it needs.

**Key takeaway:** The docstrings are what the agent reads to understand tools. A good docstring makes the difference between a tool the agent uses correctly and one it ignores. Notice how each docstring includes "Use this when..." - that guidance helps the agent decide when to call each tool.


### Step 9: Setting Up IT Operations Agent

**What we're doing:** Creating a tool registry and system prompt for the IT operations tools.

**Why:** Just like with Wikipedia search, we need to register the tools and create a system prompt that describes them to the agent.

**What to expect:**
- Create a tool registry with IT operations tools
- Create a system prompt describing the tools
- Initialize a new conversation session for IT operations


In [ ]:
# Create tool registry for IT operations tools
it_tool_registry = {
    "check_service_status": check_service_status,
    "restart_service": restart_service,
    "get_all_services": get_all_services,
}

# Create tool descriptions for system prompt
it_tool_descriptions = """
- check_service_status(service_name: str): Check the status of an IT service. 
  Returns information about service health including CPU usage, memory usage, and current status.
  Use this to monitor service health and detect issues.
  Parameters:
    - service_name (string): The name of the service to check (e.g., 'web-server', 'database', 'cache-service')

- restart_service(service_name: str): Restart an IT service that is not working properly.
  Use this when a service has failed or is in a degraded state.
  Parameters:
    - service_name (string): The name of the service to restart (e.g., 'web-server', 'database')

- get_all_services(): Get the status of all services in the environment.
  Use this to get an overview of the entire system health.
  No parameters required."""

it_system_prompt = f"""You are an IT operations agent with access to tools.

When you need to use a tool, respond ONLY with valid JSON in this exact format:
{{
  "tool_call": "function_name",
  "arguments": {{"param1": "value1"}}
}}

Available tools:
{it_tool_descriptions}

IMPORTANT: 
- If you need to use a tool, respond with ONLY the JSON object, nothing else.
- Do not include any text before or after the JSON.
- Use the exact function name as shown above.
- Always check service status before taking actions.
- Provide clear summaries of what you did."""

# Initialize conversation for IT operations
it_messages = [{"role": "system", "content": it_system_prompt}]

print("✅ IT operations tools registered!")
print(f"   Tools available: {list(it_tool_registry.keys())}")
print("\n✅ IT operations conversation initialized!")
print("   💡 The system prompt tells the agent about IT operations tools")


**What happened:** After running the code, you created a tool registry for IT operations tools and initialized a new conversation session. The system prompt describes the IT operations tools to the agent.

**Key takeaway:** You can create multiple tool registries and sessions for different purposes. Each session maintains its own context, so you can have separate conversations for different domains (Wikipedia search vs. IT operations).


### Step 10: Testing IT Operations Agent

**What we're doing:** Testing the IT operations agent with tasks that require using multiple tools in sequence.

**Why:** This demonstrates how agents can reason about problems and use multiple tools to solve them. The agent might check status first, then decide to restart a service if needed.

**What to expect:**
- Send tasks that require tool usage
- Watch the agent decide which tools to use
- See the agent use multiple tools in sequence
- Understand how agents reason about multi-step operations


In [ ]:
# Test the IT operations agent
test_tasks = [
    "Check the status of all services",
    "Check the status of cache-service and restart it if it's degraded",
]

print("=" * 60)
print("Testing IT Operations Agent")
print("=" * 60)

for i, task in enumerate(test_tasks, 1):
    print(f"\n{'=' * 60}")
    print(f"Test {i}: {task}")
    print("=" * 60)
    
    # Execute turn using helper function
    tool_used = execute_turn(task, it_messages, it_tool_registry, max_tokens=300)
    
    if tool_used:
        print("\n✅ Tool was used successfully!")
    else:
        print("\n💡 No tool call detected")
        print("   The model may have answered directly, or tool call format was different")
    
    print()

print("✅ All tests completed!")
print("\n💡 Notice how the agent reasons about which tools to use!")
print("   For the second task, it might check status first, then decide to restart.")


**What happened:** After running the tests, you saw the agent use IT operations tools to complete tasks. For the second task, the agent likely checked the service status first, detected it was degraded, and then restarted it - demonstrating multi-step reasoning!

**Key takeaway:** Agents can use multiple tools in sequence to solve complex problems. They reason about which tools to use and when, just like a human operator would. This is the power of combining AI reasoning with tools.


---

## 🎓 Key Takeaways

**What we learned:**

1. **Client-Side Tools** are Python functions with docstrings - the agent detects them automatically (no complex configuration!)
2. **Docstrings are Critical** - The agent reads docstrings to understand what tools do and when to use them
3. **Tool Registry** - A simple dictionary that maps tool names to functions
4. **Sessions** maintain conversation context - agents remember what happened in previous turns
5. **Multi-Step Operations** - Agents can use multiple tools in sequence to solve complex problems

**The big picture:**
- Tools are the agent's "hands" - they let agents take actions, not just think
- Client-side tools run in your Python process - fast, secure, no external dependencies
- The pattern is simple: function + docstring + registry = tool the agent can use
- You can build any tool you need - Wikipedia, IT operations, databases, APIs, anything!

**For IT operations:**
- Build tools for your specific infrastructure (monitoring systems, service management, etc.)
- Watch agents reason about problems and choose the right tools
- See agents execute multi-step operations (check status → detect problem → fix it)
- Create production-ready agents that can actually manage your systems

**Client-Side vs. MCP Tools:**
- **Client-Side (This Notebook):** Simple, fast, runs in your process - perfect for prototyping
- **MCP Tools (Notebook 04):** External services, standardized protocol - perfect for production

---

## 🔗 Next Steps

**Ready for more?** In **Notebook 03**, we'll explore:
- **LlamaStack Core Features** - Chat and RAG (Retrieval Augmented Generation)
- **When to use each feature** - Understanding which tool fits which problem
- **Building more powerful agents** - Combining features for better results

**The fun part:** You'll learn how to give agents access to knowledge bases (RAG) so they can answer questions about your specific infrastructure!

**Next notebook:** `03_llamastack_core_features.ipynb` - LlamaStack Core Features

**Related concepts:**
- MCP Tools for external services (covered in Notebook 04)
- Building custom MCP servers (covered in Notebook 04)
- Agent evaluation and safety (covered in Notebooks 05-06)
